# Analyzing the output of the $S$-matrix generator #

## A NOTE BEFORE STARTING ##

Since the ``prismatique`` git repository tracks this notebook under its original
basename ``analyzing_S_matrix_generator_output.ipynb``, we recommend that you
copy the original notebook and rename it to any other basename that is not one
of the original basenames that appear in the ``<root>/examples`` directory
before executing any of the notebook cells below, where ``<root>`` is the root
of the ``prismatique`` repository. This way you can explore the notebook by
executing and modifying cells without changing the original notebook, which is
being tracked by git.

## Import necessary modules ##

Upon importing the necessary modules, you may receive a warning that says
"WARNING:silx.opencl.common:Unable to import pyOpenCl. Please install it from: 
https://pypi.org/project/pyopencl". You may safely ignore this message as 
installing `pyOpenCl` is optional.

In [ ]:
# For general array handling.
import numpy as np

# For setting various visualization parameters.
import matplotlib as mpl
import matplotlib.pyplot as plt

# For loading probe model parameters.
import embeam



# The library that is the subject of this demonstration.
import prismatique

In [ ]:
# Needed in order to have interactive hyperspy plots.
%matplotlib ipympl
%matplotlib ipympl

## Introduction ##

In this notebook, we demonstrate how one can analyze select output generated by
the script ``<root>/examples/S_matrix_generator/generate.py``. Said script
generates: the $S$-matrices of the bilayer $\mathrm{MoS}_2$ sample that we
defined
[here](https://mrfitzpa.github.io/prismatique/examples/atomic_coord_generator/generate.html);
the serialized representations of the simulation parameters related to the
modelling of the sample, used to generate the $S$-matrices; the serialized
representation of the simulation parameters related to GPU and CPU workers, used
to generate the $S$-matrices; and the serialized representation of the probe
modelling parameters, used to generate the $S$-matrices. In order for the
current notebook to work properly, one must first run the aforementioned script.

You can find the documentation for the ``prismatique`` library
[here](https://mrfitzpa.github.io/prismatique/index.html).  It is recommended 
that you consult the documentation of this library as you explore the notebook.
Moreover, users should execute the cells in the order that they appear, i.e. 
from top to bottom, as some cells reference variables that are set in other 
cells above them.

## Set paths to files storing output ##

In [ ]:
# Path to file storing the S-matrices of the frozen phonon configuration subset
# #0.
path_to_S_matrix_subset_0 = \
    "../data/S_matrix_generator_output/S_matrices_of_subset_0.h5"

# Path to file storing the S-matrices of the frozen phonon configuration subset
# #1.
path_to_S_matrix_subset_1 = \
    "../data/S_matrix_generator_output/S_matrices_of_subset_1.h5"

# Path to file storing the serialized representation of the simulation
# parameters related to GPU and CPU workers, used to generate the S-matrices.
path_to_worker_params = \
    "../data/S_matrix_generator_output/worker_params.json"

# Path to file storing the serialized representation of the probe modelling
# parameters, used to generate the S-matrices.
path_to_probe_model_params = \
    "../data/S_matrix_generator_output/probe_model_params.json"

## Load simulation parameter sets and represent them as Python objects ##

In [ ]:
# Load serialized representation of the worker parameters and represent it as a
# Python object.
worker_params = prismatique.worker.Params.load(path_to_worker_params)

# Load serialized representation of the probe model parameters and represent it
# as a Python object.
cls_alias = embeam.stem.probe.ModelParams
probe_model_params = cls_alias.load(path_to_probe_model_params)

See the documentation for the ``embeam`` library
[here](https://mrfitzpa.gitlab.io/embeam/index.html) for a discussion on how to
use the Python object ``probe_model_params`` along with the ``embeam`` library
to visualize the probe model and calculate various properties of the probe.

## Extracting sample properties ##

Group the paths to the $S$-matrix files into a Python object.

In [ ]:
filenames = (path_to_S_matrix_subset_0, path_to_S_matrix_subset_1)
S_matrix_subset_ids = prismatique.sample.SMatrixSubsetIDs(filenames)

### Extract the sample unit cell dimensions in units of Å ###

In [ ]:
sample_unit_cell_dims = prismatique.sample.unit_cell_dims(S_matrix_subset_ids)

unformatted_msg = "sample unit cell dimensions (in Å): {}"
msg = unformatted_msg.format(sample_unit_cell_dims)
print(msg)

### Extract the sample supercell dimensions in units of Å ###

In [ ]:
sample_supercell_dims = prismatique.sample.supercell_dims(S_matrix_subset_ids)

unformatted_msg = "sample supercell dimensions (in Å): {}"
msg = unformatted_msg.format(sample_supercell_dims)
print(msg)

### Extract the lateral dimensions of the sample supercell in units of pixels ###

In [ ]:
sample_supercell_xy_dims_in_pixels = \
    prismatique.sample.supercell_xy_dims_in_pixels(S_matrix_subset_ids)

unformatted_msg = "lateral dimensions of sample supercell (in pixels): {}"
msg = unformatted_msg.format(sample_supercell_xy_dims_in_pixels)
print(msg)

### Extract the lateral pixel size of the sample supercell in units of Å ###

In [ ]:
sample_supercell_lateral_pixel_size = \
    prismatique.sample.supercell_lateral_pixel_size(S_matrix_subset_ids)

unformatted_msg = "sample supercell lateral pixel size (in Å): {}"
msg = unformatted_msg.format(sample_supercell_lateral_pixel_size)
print(msg)

### Extract the number of frozen phonon configuration subsets ###

In [ ]:
num_frozen_phonon_config_subsets = \
    prismatique.sample.num_frozen_phonon_config_subsets(S_matrix_subset_ids)

unformatted_msg = "# frozen phonon configuration subsets: {}"
msg = unformatted_msg.format(num_frozen_phonon_config_subsets)
print(msg)

### Extract the number of frozen phonon configurations in each subset ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.num_frozen_phonon_configs_in_subset``:

In [ ]:
func_alias = prismatique.load.num_frozen_phonon_configs_in_subset
filenames = S_matrix_subset_ids.core_attrs["filenames"]

for subset_idx, filename in enumerate(filenames):
    num_frozen_phonon_configs_in_subset = func_alias(filename)
    
    unformatted_msg = "# frozen phonon configurations in subset {}: {}"
    msg = unformatted_msg.format(subset_idx, 
                                 num_frozen_phonon_configs_in_subset)
    print(msg)

The second method involves the use of the function
``prismatique.sample.num_frozen_phonon_configs_in_subset``:

In [ ]:
func_alias = prismatique.sample.num_frozen_phonon_configs_in_subset

for subset_idx in range(num_frozen_phonon_config_subsets):
    num_frozen_phonon_configs_in_subset = func_alias(S_matrix_subset_ids,
                                                     subset_idx)
    
    unformatted_msg = "# frozen phonon configurations in subset {}: {}"
    msg = unformatted_msg.format(subset_idx, 
                                 num_frozen_phonon_configs_in_subset)
    print(msg)

### Extract the total number of frozen phonon configurations ###

In [ ]:
total_num_frozen_phonon_configs = \
    prismatique.sample.total_num_frozen_phonon_configs(S_matrix_subset_ids)

unformatted_msg = "total # of frozen phonon configurations: {}"
msg = unformatted_msg.format(total_num_frozen_phonon_configs)
print(msg)

## Loading $S$-matrix wavefunctions ##

In the code blocks below, we load a subcollection of the $S$-matrix
wavefunctions of one subset.

Load a $S$-matrix wavefunction subcollection into a ``hyperspy`` signal.

In [ ]:
kwargs = {"filename": path_to_S_matrix_subset_0, 
          "multi_dim_slice": ([2, 3], slice(1, 8, 2))}

S_matrix_wavefunction_signal, navigational_to_original_indices_map = \
    prismatique.load.S_matrix_wavefunctions(**kwargs)

``navigational_to_original_indices_map`` is a dictionary that maps the
navigational indices of the hyperspy signal ``S_matrix_wavefunction_signal`` to
the original indices specified by ``multi_dim_slice``. For example, if the
original atomic configuration indices map to a set of corresponding navigational
indices, then
``navigational_to_original_indices_map["atomic_config_indices"][i]`` yields the
atomic configuration index specified in the expression
``single_dim_slice=multi_dim_slice[0] if multi_dim_slice is not None else
slice(None)`` that corresponds to the ``i`` th atomic configuration index in the
nagivation index space of ``S_matrix_wavefunction_signal``, where ``i`` is a
nonnegative integer smaller than the total number of atomic configuration
indices specified in ``single_dim_slice``.

In [ ]:
navigational_to_original_indices_map

Show the $S$-matrix wavefunction signal metadata.

In [ ]:
S_matrix_wavefunction_signal.metadata

Plot the $S$-matrix wavefunction subcollection.

In [ ]:
kwargs = {"colorbar": True,
          "scalebar": True,
          "axes_ticks": True,
          "gamma": 1,
          "cmap": plt.get_cmap("jet")}
S_matrix_wavefunction_signal.plot(representation="polar", **kwargs)